In [60]:
import preprocess
import json
import glob
from keras.models import Sequential
import keras
from keras.layers import Merge, LSTM, Dense,GRU
from keras.preprocessing import sequence
from keras.layers.wrappers import Bidirectional
import numpy as np
import pickle
from sklearn.cluster import KMeans,DBSCAN
from sklearn.neighbors import KNeighborsClassifier
from pprint import pprint
from sklearn.externals import joblib
from BDkeras import Classifier

In [61]:
pklpath = "./models/test10.pickle"
name = pklpath.split("/")[-1].replace(".pickle","")
#pklpath = sys.argv[1]
pkl = pickle.load(open(pklpath,"rb"))
#pad_size = pkl["pad_size"]
pad_size = 30
pcapath = pkl["pcapath"]
knnpath = pkl["knnpath"].replace("knn","kmeans")
pca = joblib.load(pcapath)
knn = joblib.load(knnpath)
from gensim.models import word2vec
w2vpath='./w2v.model'
w2v = word2vec.Word2Vec.load(w2vpath)
clf = Classifier(pad_size=pad_size,wvdim=200).decoder

/home/takayama/research/DBDC/bd_full/BDkeras.py:29: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(batch_input_shape=(None, 30,..., return_sequences=False, recurrent_dropout=0.8, units=200)`
  encoder_a.add(Bidirectional(GRU(output_dim=wvdim, batch_input_shape=(None, pad_size, wvdim), return_sequences=False,dropout_U=0.8), batch_input_shape=(None, pad_size, wvdim)))
/home/takayama/research/DBDC/bd_full/BDkeras.py:33: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(batch_input_shape=(None, 30,..., return_sequences=False, recurrent_dropout=0.8, units=200)`
  encoder_b.add(Bidirectional(GRU(output_dim=wvdim, batch_input_shape=(None, pad_size, wvdim), return_sequences=False,dropout_U=0.8), batch_input_shape=(None, pad_size, wvdim)))
/home/takayama/research/DBDC/bd_full/BDkeras.py:37: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  decoder.add(Merge([enc

In [62]:
cluster_sent = [[] for i in range(10)]
res_v = []
cnt_l = {
    domain+"_"+str(i):0 for i in range(10) for domain in ["DCM","DIT","IRS"]
}
for domain in ["DCM","DIT","IRS"]:
    for fp in glob.glob("../DBDC2_ref/"+domain+"/*.json"):
        tmp = fp.split("/")
        dataID = tmp[-1].strip(".log.json")
        #domain = tmp[-2]
        result = {"dialogue-id":dataID,"turns":[]}
        aaa = preprocess.corpusGenerator([fp])
        rnn_vec,knn_vec= preprocess.vectorize(preprocess.tokenize(aaa),w2v=w2v)
        (xu_list,xs_list,y_list) = zip(*rnn_vec)
        (xu_cl,xs_cl,_) = zip(*knn_vec)
        xu_list = sequence.pad_sequences(xu_list,pad_size)
        xs_list = sequence.pad_sequences(xs_list,pad_size)
        #labels = knn.predict(np.c_[np.sum(xu_list,axis=1),np.sum(xs_list,axis=1)])
        #labels = knn.predict(np.average(xu_list,axis=1)-np.average(xs_list,axis=1))
        labels = knn.predict(pca.transform(xu_cl))
        #print(labels)
        for lbl in labels:
            cnt_l[domain+"_"+str(lbl)] += 1
        for i,zipped in enumerate(zip(aaa,xu_list,xs_list,labels)):
            bbb,xu,xs,label = zipped
            cluster_sent[label].append((bbb[:2],domain)) #あとで消す
            turnIndex = i*2
            clf.load_weights(pkl["modelpath"][label])
            predict = clf.predict([np.array([xu]),np.array([xs])])[0]
            probO,probT,probX = predict
            breakdown = ["O","T","X"][predict.argmax()]
            result["turns"].append({
                "turn-index":turnIndex,
                "labels":[{
                    "breakdown":breakdown,
                    "prob-O":float(probO),
                    "prob-T":float(probT),
                    "prob-X":float(probX),
                }]
            })
            res_v.append((bbb,probO,probT,probX))
        json.dump(result,open("./result/"+domain+"/"+dataID+".labels.json","w"))

In [63]:
cnt_l

{'DCM_0': 49,
 'DCM_1': 96,
 'DCM_2': 48,
 'DCM_3': 6,
 'DCM_4': 185,
 'DCM_5': 19,
 'DCM_6': 23,
 'DCM_7': 17,
 'DCM_8': 59,
 'DCM_9': 48,
 'DIT_0': 79,
 'DIT_1': 51,
 'DIT_2': 63,
 'DIT_3': 23,
 'DIT_4': 129,
 'DIT_5': 33,
 'DIT_6': 28,
 'DIT_7': 46,
 'DIT_8': 62,
 'DIT_9': 36,
 'IRS_0': 77,
 'IRS_1': 63,
 'IRS_2': 50,
 'IRS_3': 24,
 'IRS_4': 115,
 'IRS_5': 23,
 'IRS_6': 27,
 'IRS_7': 38,
 'IRS_8': 73,
 'IRS_9': 60}

In [57]:
cnt_l

{'DCM_0': 53,
 'DCM_1': 97,
 'DCM_2': 49,
 'DCM_3': 4,
 'DCM_4': 189,
 'DCM_5': 15,
 'DCM_6': 13,
 'DCM_7': 17,
 'DCM_8': 67,
 'DCM_9': 46,
 'DIT_0': 85,
 'DIT_1': 55,
 'DIT_2': 69,
 'DIT_3': 18,
 'DIT_4': 130,
 'DIT_5': 28,
 'DIT_6': 29,
 'DIT_7': 44,
 'DIT_8': 55,
 'DIT_9': 37,
 'IRS_0': 85,
 'IRS_1': 64,
 'IRS_2': 63,
 'IRS_3': 21,
 'IRS_4': 113,
 'IRS_5': 16,
 'IRS_6': 13,
 'IRS_7': 29,
 'IRS_8': 80,
 'IRS_9': 66}

In [12]:
cluster_sent = [[] for i in range(10)]

aaa = [(input(),input(),[1,0,0])]
xu_list,xs_list,y_list = zip(*preprocess.vectorize(preprocess.tokenize(aaa),w2v=w2v))
xu_list = sequence.pad_sequences(xu_list,pad_size)
xs_list = sequence.pad_sequences(xs_list,pad_size)
#labels = knn.predict(np.c_[np.sum(xu_list,axis=1),np.sum(xs_list,axis=1)])
#labels = knn.predict(np.average(xu_list,axis=1)-np.average(xs_list,axis=1))
labels = knn.predict(np.sum(xu_list,axis=1)/np.linalg.norm(xu_list,axis=1))
print(labels)
for i,zipped in enumerate(zip(aaa,xu_list,xs_list,labels)):
    bbb,xu,xs,label = zipped
    turnIndex = i*2
    clf.load_weights(pkl["modelpath"][label])
    predict = clf.predict([np.array([xu]),np.array([xs])])[0]
    probO,probT,probX = predict
    breakdown = ["O","T","X"][predict.argmax()]
    print(probO,probT,probX)

/home/takayama/.local/lib/python3.5/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':
/home/takayama/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [16]:
#システム側が頑張って話が続くようにしてる感じがするクラスタ
for line in cluster_sent[0]:
        print("user  :",line[0][0])
        print(line[1]+"  :",line[0][1])
        print()

user  : 
DCM  : こんにちは。声かけてくれるのを待ってたんだ。

user  : 今いろんな映画やっていますもんね
DCM  : 映画は大丈夫です

user  : 映画でポップコーン食べるのが好きです。
DCM  : 映画は元気ですね

user  : 楽しみですね。
DCM  : 映画は楽しみですか。そうですね

user  : ゴジラが気になっているよ
DCM  : 映画は一人がいいですね

user  : 音楽好きなんですか？
DCM  : 映画は映画館がいいですね

user  : 
DCM  : こんばんは。声かけてくれるのを待ってたんだ。

user  : こんばんは。それは嬉しいです。
DCM  : 今日の夕食は済みました？

user  : カレーライスを食べました。
DCM  : 夕食は無しか？？

user  : 食べました。
DCM  : 食べてない

user  : まだなんですね。
DCM  : 晩御飯を食べるかもしれない

user  : 何が食べたいですか?
DCM  : 御飯を食べに行きますよねえ

user  : じゃあどんな食べ物が好きですか?
DCM  : 炊きたてがおいしいですね

user  : ご飯は炊きたてが美味しいですね。
DCM  : どんなに落ち込んでいても、おいしいご飯を食べると元気になるよね。

user  : ご飯がとても好きなようですね。
DCM  : 食べるのがいいですね

user  : 
DCM  : こんにちは！夏といえばスイカだね。

user  : こんにちは、スイカ好きなんだね
DCM  : スイカで有名か？？

user  : 近くに産地があるよ
DCM  : イタリアが好きですね

user  : 大好きだよ
DCM  : イタリア料理は大好きですか。もう一回言ってください

user  : イタリア料理は大好き
DCM  : イタリア料理は大好きですね

user  : うん、好きでよく食べるよ
DCM  : イタリア料理で有名ですね

user  : そうなんだ。例えばなんてお店??
DCM  : イタリア料理は美味しいですね

user  : 私はパスタが好きだよ
DCM  : イタリア料理は美味しいですよねー

user  : ピザも好きだな
DCM  : イタリア

IRS  : あぁ！たまにアニメ見てました。でもストーリーがいまいち掴めなくて…私は放置しちゃいましたよ。

user  : そういうことありますよね〜。
IRS  : そうです。後ろにいるような感じしません？想像力豊かなので。

user  : ところで、最近暑い日が続いていますね
IRS  : そうですね。風邪とかは引いていませんか？

user  : いいえ。年に二回ぐらい。
IRS  : そうなんですか。よくあるんですかこのバイト？

user  : もう直ったので大丈夫ですよ。
IRS  : すみません。急いでアップしてしまいました。

user  : 
IRS  : こんにちは。好きな動物は何ですか？

user  : 猫が好きです
IRS  : どんな種類の猫なんですか？

user  : 日本の猫。茶色も黒猫もかわいいですね
IRS  : うちのチビは冷たいですね。名前呼んでも無視するくせに、エサの時は近づいてきますから。

user  : そういうところがまたかわいい
IRS  : かわいいというか…ドジですね。そういってくれるTEFS15さんは、優しいです。

user  : 動物園とか水族館とか行きたいですね
IRS  : ぜひ行って下さい！安いしデートにも最適！

user  : デートに水族館は盛り上がりますね
IRS  : 水族館いいね！私彼氏と水族館行ったことない！花火大会とかお祭り行きたいな。

user  : 春の花見と夏の花火はやっぱりいいよね
IRS  : ですです！冬は寒いですものね

user  : 冬は自宅で正月がいいですね
IRS  : ペンギンやシロクマも過ごしやすいでしょうね。

user  : 
IRS  : こんにちは。音楽はどんなのを聴かれていますか？

user  : アニソンが多いです。
IRS  : 他にはどんな曲を弾くの？

user  : 楽器はまったく弾きませんね。
IRS  : この夏は花火大会や海にお出かけになられましたか。

user  : では、今は何を？
IRS  : 今はグッドだけですよ〜

user  : それはよかったですね。
IRS  : さっきのドライブの話に戻りますけど

user  : はい。
IRS  : 木の素材とか、大量生産してるとか、手作りとか・・そういうので全然値段

In [17]:
#とりあえず同じキーワードが入った文を適当に投げてる感じのクラスタ
for line in cluster_sent[1]:
        print("user  :",line[0][0])
        print(line[1]+"  :",line[0][1])
        print()

user  : こんにちは。最近興味のあることってありますか？
DCM  : 興味が湧いてないかもです

user  : そうですかー。映画とか気になるのありません？
DCM  : 映画が楽しみです

user  : 私も元気な映画を見るのは好きですよ
DCM  : 映画が楽しみですね

user  : 他にもお祭りや花火も楽しみだよ
DCM  : 映画が良いですね

user  : 一人で見るほうが気楽でいいですよね
DCM  : 映画は音楽がいいですね

user  : 夕食は食べてないのですか?
DCM  : 実家で食べるのです

user  : そうかもしれません。好きな食べ物はご飯ですか?
DCM  : どんなに落ち込んでいても、おいしいご飯を食べると元気になるよね。

user  : 行ったことないよ。イタリア料理は好きだな
DCM  : イタリア料理は好きですね

user  : そんなことはないんじゃないかな？
DCM  : そんなことある

user  : まだ今年は海へ行ってませんね。
DCM  : 海に行きませんか？？

user  : たまには遠出するのも悪くありません。
DCM  : 遠出は好きですね

user  : 新しい発見があるかもしれませんからね。
DCM  : 遠出は楽しいですね

user  : その地域のお祭りなどにも行きたいですね。
DCM  : お祭りに行きたいんですね？お祭りは夜が楽しいですよねえ

user  : 夜のお祭りも風情があっていいですからね。
DCM  : 風情は欲しいですね

user  : 持っているお金の範囲で
DCM  : 趣味は本業にしない方がいいって真実だよね。

user  : こんにちは、あなたはスイカは好きなんですか？
DCM  : スイカで有名ですね

user  : スイカで何が有名なんですか？
DCM  : スイカはうまいらしいです

user  : 他に好きなフルーツはなんですか？
DCM  : スイカに塩じゃなくて、砂糖をかけて食べる国もあるんだって。

user  : 甘いものといえば、お菓子は好きですか？
DCM  : 少な目がいいですね

user  : じゃあダイエットしているのですか？
DCM  : ダイエットで稼ぐのです

user  : 好きな運動は何ですか？
D

In [85]:
for line in cluster_sent[2]:
        print("user  :",line[0][0])
        print(line[1]+"  :",line[0][1])
        print()

user  : こんにちは。最近興味のあることってありますか？
DCM  : 興味が湧いてないかもです

user  : 今いろんな映画やっていますもんね
DCM  : 映画は大丈夫です

user  : 映画でポップコーン食べるのが好きです。
DCM  : 映画は元気ですね

user  : 楽しみですね。
DCM  : 映画は楽しみですか。そうですね

user  : 他にもお祭りや花火も楽しみだよ
DCM  : 映画が良いですね

user  : ゴジラが気になっているよ
DCM  : 映画は一人がいいですね

user  : 音楽好きなんですか？
DCM  : 映画は映画館がいいですね

user  : カレーライスを食べました。
DCM  : 夕食は無しか？？

user  : 食べました。
DCM  : 食べてない

user  : 夕食は食べてないのですか?
DCM  : 実家で食べるのです

user  : まだなんですね。
DCM  : 晩御飯を食べるかもしれない

user  : 何が食べたいですか?
DCM  : 御飯を食べに行きますよねえ

user  : じゃあどんな食べ物が好きですか?
DCM  : 炊きたてがおいしいですね

user  : ご飯がとても好きなようですね。
DCM  : 食べるのがいいですね

user  : こんにちは、スイカ好きなんだね
DCM  : スイカで有名か？？

user  : 近くに産地があるよ
DCM  : イタリアが好きですね

user  : イタリア料理は大好き
DCM  : イタリア料理は大好きですね

user  : うん、好きでよく食べるよ
DCM  : イタリア料理で有名ですね

user  : 私はパスタが好きだよ
DCM  : イタリア料理は美味しいですよねー

user  : ピザも好きだな
DCM  : イタリア料理は得意ですね

user  : まだ今年は海へ行ってませんね。
DCM  : 海に行きませんか？？

user  : いいですね。
DCM  : いいんですか

user  : たまには遠出するのも悪くありません。
DCM  : 遠出は好きですね

user  : その地域のお祭りなどにも行きたいですね。
DCM  : お祭りに行きたいんですね？お祭りは夜が楽しいですよ

In [18]:
for line in cluster_sent[3]:
        print("user  :",line[0][0])
        print(line[1]+"  :",line[0][1])
        print()

In [87]:
for line in cluster_sent[4]:
        print("user  :",line[0][0])
        print(line[1]+"  :",line[0][1])
        print()

user  : 
DCM  : こんにちは！夏といえばスイカだね。

user  : 大好きだよ
DCM  : イタリア料理は大好きですか。もう一回言ってください

user  : 
DCM  : こんにちは！海へ行きたいね。

user  : やめてください
DCM  : どんな趣味をお持ちですか？

user  : かかりますねー
DCM  : お金はかかってるんですね。趣味を広げるんですか？

user  : 持っているお金の範囲で
DCM  : 趣味は本業にしない方がいいって真実だよね。

user  : 
DCM  : こんばんは！夏といえばスイカだね。

user  : 
DCM  : こんにちはー。海へ行きたいね。

user  : こんにちは、暑いですからね。
DCM  : 海に行きませんか？？

user  : 
DCM  : こんにちはー。海へ行きたいね。

user  : 
DCM  : こんにちは。夏といえばスイカだね。

user  : こんにちは。暑いですね。
DCM  : スイカで買い物できてるのですか？

user  : 買い物ちょっと大変ですけどね。
DCM  : 休日のお買い物は、心が弾んで、つい買い過ぎちゃうよね。

user  : 
DCM  : こんばんは！夏といえばスイカだね。

user  : こんにちは。暑いですね。
DCM  : ところで、気になるニュースがあったのですが、7月29日から公開された映画「シン・ゴジラ」の観客数が1日までに71万人に達し、1954年に第1作が公開された「ゴジラ」シリーズの累計観客数が1億人突破し、実写映画では初めてだそうですよ。

user  : 気をつけていますよ。水分を多めに摂っています。
DCM  : 熱中症に気をつけてるんですね。熱中症に気をつけか？？

user  : こんにちわ。毎日暑いよね。
DCM  : 熱中症に気をつけないんですか？

user  : 気をつけます。ありがとう。
DCM  : 熱中症に気をつけか？？

user  : 海に行きました
DCM  : 海に行かないのですか

user  : 夏になったら必ず行きたくなります
DCM  : 夏になってるんですね。海は透明度が良いですね

user  : 日々努力ですね。
DCM  : 女子力UPで目指してますか